In [ ]:

library(data.table)
r.posts  <- fread("posts.csv",na.strings = c("", " ", "NA", "NULL", "N/A", "NaN"))
r.replys  <- fread("replys.csv",na.strings = c("", " ", "NA", "NULL", "N/A", "NaN"))
r.posts
r.replys

In [ ]:
r.posts <- r.posts[postnums!=0,]
posts_id <-r.posts[,.N]
posts_id


In [ ]:
lwg_post_count <- r.posts[guba_name == "螺纹钢吧", .N]
lwg_post_count<posts_id

In [ ]:
setorder(r.posts, create_time,na.last = FALSE)
early=r.posts[,.SD[1],by=.(guba_name)]
early
fwrite(early,'earl.posts.csv')


In [ ]:
setorder(r.replys, reply_time,na.last = FALSE)
late=r.replys[,.SD[.N],by=.(post_id)]
late
fwrite(late,'late.replys.csv')

In [ ]:
library(stringr)
r.replys[, reply_content := str_remove_all(reply_content, "<[^>]+>")]
r.replys$clean_content=NULL
r.replys

In [ ]:
int.data=test=merge(r.posts,r.replys,by='post_id',all=FALSE)
int.data

In [ ]:
int.data[,create_date:=as.Date(create_time, format = "%Y-%m-%d")]
int.data
int.data[,create_date:=as.POSIXct(create_time, format = "%Y-%m-%d %H:%M:%OS")]
int.data

In [ ]:
maxday=int.data[,max(create_date)]
minday=int.data[,min(create_date)]
jiuba=int.data[,guba_name]
list_date=seq(from = minday, to = maxday, by = "days")
tim=CJ(guba_name=jiuba,create_date=list_date)
tim

In [ ]:
int.data.ts=merge(tim,int.data,by=c('guba_name','create_date'),all =FALSE)
# 填充 readnum 列的缺失值
int.data.ts[is.na(readnum), readnum := 0]
# 填充 replynum 列的缺失值
int.data.ts[is.na(replynum), replynum := 0]
int.data.ts

In [ ]:
max_notice=r.posts[order(-replynum)][1:10,.(post_id, guba_name, readnum, replynum)]
max_notice
fwrite(max_notice,'mx.post.csv')
max_jiuba=r.posts[,.(total_readnum=sum(readnum), total_replynum=sum(replynum)),by=.(guba_name)][order(-total_replynum)][1:10]
max_jiuba
fwrite(max_jiuba,'mx.ba.csv')

In [ ]:
r.posts=r.posts[!is.na(readnum)&!is.na(replynum)]
test_one=r.posts[, .(
  replynum_min = min(replynum),
  replynum_max = max(replynum),
  replynum_mean = mean(replynum),
  replynum_median = median(replynum),
  replynum_var = var(replynum),
  readnum_min = min(readnum),
  readnum_max = max(readnum),
  readnum_mean = mean(readnum),
  readnum_median = median(readnum),
  readnum_var = var(readnum)
)]
stst.int <- data.table(
  variable = c("replynum", "readnum"),
  min = c(test_one$replynum_min, test_one$readnum_min),
  max = c(test_one$replynum_max, test_one$readnum_max),
  mean = c(test_one$replynum_mean, test_one$readnum_mean),
  median = c(test_one$replynum_median, test_one$readnum_median),
  variance = c(test_one$replynum_var, test_one$readnum_var)
)
print(stst.int)
fwrite(stst.int,'stat.int.csv')